# 3D Rotation
3D Rotation can be represented in different ways as discussed in last week's lectures.
These include:
1.	Rotation matrices
2.  Three angle representation
3.	Two vector representation
4.	Rotation about an eigen vector
5.	Unit quaternion
6.	Matrix exponential

We explored the `rotation matrices` approach in last weeks lab exercise.

This lab is designed for you to implement rotation matrices in each of those different methods.

In [12]:
# import required libraries
import numpy as np
import matplotlib.pyplot as plt
# To render plots inline
%matplotlib inline

# For unit tests
from tester import LabTester
lab_tester = LabTester()

In [13]:
def rotation_three_angle(alpha:float, gamma:float, theta:float, p:np.ndarray) -> np.ndarray:
    """
    Input:
        alpha, gamma, theta - Euler angles of rotation about the axes (x, y, x)
        p                   - The vector to be transformed
    Output: resulting coordinates after appyling the rotation
    """
    # Your code here - Use Euler angles method => (x,y,x)
    # Rotation matrix about x-axis
    def Rx(angle):
        return np.array([
            [1, 0, 0],
            [0, np.cos(angle), -np.sin(angle)],
            [0, np.sin(angle), np.cos(angle)]
        ])
    
    # Rotation matrix about y-axis
    def Ry(angle):
        return np.array([
            [np.cos(angle), 0, np.sin(angle)],
            [0, 1, 0],
            [-np.sin(angle), 0, np.cos(angle)]
        ])
        
    Rx1 = Rx(np.deg2rad(alpha))
    Ry1 = Ry(np.deg2rad(gamma))
    Rx2 = Rx(np.deg2rad(theta))
    
    # Apply the transformations in sequence (x -> y -> x)
    p =  Rx1 @ Ry1 @ Rx2 @ p
    return p

In [14]:
# Run test cases
lab_tester.test_rotation_three_angle(rotation_three_angle)


=== Testing Three Angle Rotation Implementation ===
✅ No Rotation
✅ 45 Degree Rotation
✅ Full Rotation
✅ Negative 45 Degree Rotation


In [15]:
def rotation_two_vector(a_v: np.ndarray, o_v: np.ndarray, p:np.ndarray) -> np.ndarray:
    """
    Input:
        a_v     - Approach vector
        o_v     - Orientation vector
        p       - The vector to be transformed
    Output: resulting coordinates after appyling the rotation
    """
    # Your code here
    o = o_v / np.linalg.norm(o_v)
    a = a_v / np.linalg.norm(a_v)

    # Compute normal vector
    n = np.cross(o, a)
    if np.linalg.norm(n) < 1e-10:
        raise ValueError("error")
    n /= np.linalg.norm(n)

    # Recompute orthogonal orientation vector
    o_prime = np.cross(a, n)
    o_prime /= np.linalg.norm(o_prime)

    # Construct rotation matrix
    R = np.column_stack((n, o_prime, a))

    P = np.round(R @ p, 3)
    
    return P

In [16]:
lab_tester.test_rotation_two_vector(rotation_two_vector)


=== Testing Two Vector Rotation Implementation ===
✅ X-Axis Rotation
✅ Y-Axis Rotation
✅ Z-Axis Rotation
✅ 45-degree Z Rotation


In [17]:
def rotation_eigen_vector(neta: np.ndarray, theta:float, p:np.ndarray) -> np.ndarray:
    """
    Input:
        neta    - Eigen vector with eigen value 1
        theta   - Angle of rotation about neta
        p       - The vector to be transformed
    Output: resulting coordinates after appyling the rotation
    """
    # Your code here - Use Rodirgues Formula
    # Ensure the eigenvector is a unit vector
    neta = neta / np.linalg.norm(neta)

    # Compute Rodrigues' rotation formula
    p = (
        p * np.cos(np.deg2rad(theta)) +
        np.cross(neta, p) * np.sin(np.deg2rad(theta)) +
        neta * np.dot(neta, p) * (1 - np.cos(np.deg2rad(theta)))
    )
    return p

In [18]:
lab_tester.test_rotation_eigen_vector(rotation_eigen_vector)


=== Testing Eigen Vector Rotation Implementation ===
✅ No Rotation
✅ 180-degree Z Rotation
✅ 90-degree X Rotation
✅ 45-degree Y Rotation
✅ Negative 90-degree Z Rotation


In [19]:
def rotation_unit_quaternion(q: np.ndarray, p: np.ndarray) -> np.ndarray:
    # Your code here - q: Unit quaternion vector [Real, i, j, k]
    # Extract components of quaternion
    q0, q1, q2, q3 = q
    px, py, pz = p

    # Compute the quaternion product q * P * q_conj
    P = np.array([0, px, py, pz])  
    q_conj = np.array([q0, -q1, -q2, -q3])  

    # Quaternion multiplication function
    def quat_mult(a, b):
        """Multiplies two quaternions a and b."""
        w1, x1, y1, z1 = a
        w2, x2, y2, z2 = b
        return np.array([
            w1*w2 - x1*x2 - y1*y2 - z1*z2,
            w1*x2 + x1*w2 + y1*z2 - z1*y2,
            w1*y2 - x1*z2 + y1*w2 + z1*x2,
            w1*z2 + x1*y2 - y1*x2 + z1*w2
        ])

    # Perform rotation: P' = q * P * q_conj
    P_rotated = quat_mult(quat_mult(q, P), q_conj)

    # Return the rotated vector part
    return P_rotated[1:]  

# Example usage
q = np.array([0.7071, 0, 0.7071, 0])  
p = np.array([1, 0, 0])  

rotated_p = rotation_unit_quaternion(q, p)
print(rotated_p)  

[ 0.          0.         -0.99998082]


In [20]:
lab_tester.test_rotation_unit_quaternion(rotation_unit_quaternion)


=== Testing Unit Quaternion Rotation Implementation ===
✅ No Rotation
✅ 90-degree X Rotation
✅ 180-degree Y Rotation
✅ 45-degree Z Rotation
✅ Negative 90-degree X Rotation


In [21]:
def skew_symmetric(w_x: np.ndarray) -> np.ndarray:
    """Returns the skew-symmetric matrix of a 3D vector w_x."""
    return np.array([
        [0, -w_x[2], w_x[1]],
        [w_x[2], 0, -w_x[0]],
        [-w_x[1], w_x[0], 0]
    ])

def rotation_matrix_exp(w_x: np.ndarray, theta: float, p: np.ndarray) -> np.ndarray:
    """
    Rotate point p using the matrix exponential of the skew-symmetric matrix of w_x.
    
    Parameters:
        w_x : np.ndarray - The unit vector representing the axis of rotation (3D).
        theta : float - The angle of rotation in **degrees**.
        p : np.ndarray - The point to be rotated (3D).
        
    Returns:
        np.ndarray - The rotated point (3D).
    """
    # Ensure w_x is a unit vector
    w_x = w_x / np.linalg.norm(w_x)

    # Convert angle to radians
    theta_rad = np.radians(theta)

    # Compute the skew-symmetric matrix of w_x
    W_x = skew_symmetric(w_x)
    
    # Calculate the rotation matrix using Rodrigues' formula
    R = np.eye(3) + np.sin(theta_rad) * W_x + (1 - np.cos(theta_rad)) * np.dot(W_x, W_x)
    
    # Apply the rotation to the point
    p_rotated = np.dot(R, p)

    ans = np.round(p_rotated, 3)
    
    # Round to avoid floating-point precision issues
    return ans


In [22]:
lab_tester.test_rotation_matrix_exp(rotation_matrix_exp)


=== Testing Matrix Exponential Rotation Implementation ===
✅ No Rotation
✅ 90-degree X-axis Rotation
✅ 90-degree Y-axis Rotation
✅ 90-degree Z-axis Rotation
✅ 180-degree X-axis Rotation
✅ 180-degree Y-axis Rotation
✅ 180-degree Z-axis Rotation
✅ 45-degree X-axis Rotation
